In [2]:
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd


options = webdriver.ChromeOptions()
options.add_argument("headless")

In [3]:
driver_path = 'C:/Users/MY/Downloads/chromedriver.exe'

In [4]:
def clean_text(text):
    cleaned_text = re.sub('[a-zA-Z]', '', text)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                          '', cleaned_text)
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

In [7]:
def biz_khan(query,driver_path,page):
    domain = '서울신문'
    
    current_page = 1
    last_page =(int(page)-1)*10+1
    
    head_list = []
    data_list = []
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path=driver_path)
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain ='%EB%89%B4%EC%8A%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1081&nso=&is_sug_officeid=0' 
        url = url_domain + query + '&%2Ca%3A&start='+str(current_page) + news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        
        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
                
        except:
            print('기사가 없')
            driver.close()
            return 0
        driver.close()
        
        
        for url in url_list:
            driver = webdriver.Chrome(executable_path=driver_path)  # for Windows
            driver.get(url)
            time.sleep(2)
                
            try:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(soup.find('h1',class_ = 'atit2').text) #class_ = 'atit2'
                tags = soup.select_one('#atic_txt1').find_all()
                data = [x.text for x in tags]
                data_list.append(' '.join(data))
            
            except:
                try:
                    soup = BeautifulSoup(driver.page_source,'html.parser')
                    head_list.append(soup.find('h1',class_="articleTitleDiv").text)
                    tags = soup.select_one('body > div.middleWrap > div > div.mLeftWrap > div.articleDiv').find_all()
                    data = [x.text for x in tags]
                    data_list.append(' '.join(data))
                    
                except:
                    driver.close()
                current_page+=10
        
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

biz = biz_khan('풀무원 바른먹거리',driver_path,2)
biz

,언론사,제목,내용
0,서울신문,풀무원 ‘바른먹거리 캠페인’ 확대…올해 어린이 1만명에 식생활 교육,\n\n\n\n\n\n\n\n\t\t\t $(window).load(f...
1,서울신문,[새 광고] 꼼꼼한 주부의 바른먹거리 삼매경,
2,서울신문,"풀무원, 청주 오송에 연구소 짓는다.",\n\n\n\n\n\n\n\n\t\t\t $(window).load...
3,서울신문,"[뉴스 플러스] 풀무원, 베이징에 첫 식음료 매장 개장",\n\n\n\n\n\n\n \n\n\n\n\n\n \n\n\n\n\n \n\n\n
4,서울신문,[사회공헌] 풀무원-매출액 0.1% 아토피 어린이 지원,\n\n\n\n\n\n\n\n\t\t\t $(window).load(f...
5,서울신문,붉은 먹거리가 뜬다,선인장 열매즙·딸기 도넛 등 잇따라 출시 \n\n\n\n\n\n\n\n\t\t...


In [8]:
biz.to_csv('서울신문.csv',index=None)

In [9]:
data1 = pd.read_csv('서울신문.csv')

In [10]:
data1

,언론사,제목,내용
0,서울신문,풀무원 ‘바른먹거리 캠페인’ 확대…올해 어린이 1만명에 식생활 교육,\n\n\n\n\n\n\n\n\t\t\t $(window).load(f...
1,서울신문,[새 광고] 꼼꼼한 주부의 바른먹거리 삼매경,NaN
2,서울신문,"풀무원, 청주 오송에 연구소 짓는다.",\n\n\n\n\n\n\n\n\t\t\t $(window).load...
3,서울신문,"[뉴스 플러스] 풀무원, 베이징에 첫 식음료 매장 개장",\n\n\n\n\n\n\n \n\n\n\n\n\n \n\n\n\n\n \n\n\n
4,서울신문,[사회공헌] 풀무원-매출액 0.1% 아토피 어린이 지원,\n\n\n\n\n\n\n\n\t\t\t $(window).load(f...
5,서울신문,붉은 먹거리가 뜬다,선인장 열매즙·딸기 도넛 등 잇따라 출시 \n\n\n\n\n\n\n\n\t\t...
